In [ ]:
!pip install torch scikit-learn matplotlib shap pandas numpy


In [ ]:
import torch

import torch.nn as nn

import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np

import pandas as pd

import re

import matplotlib.pyplot as plt



# Custom CNN for Text Classification

class CNN_Text(nn.Module):

    def __init__(self, vocab_size, embed_size, num_classes, kernel_sizes, num_filters, dropout=0.5):

        super(CNN_Text, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size)

        self.convs = nn.ModuleList([

            nn.Conv2d(1, num_filters, (kernel_size, embed_size)) for kernel_size in kernel_sizes

        ])

        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)



    def forward(self, x):

        x = self.embedding(x)  # [batch_size, seq_len, embed_size]

        x = x.unsqueeze(1)  # Add a channel dimension: [batch_size, 1, seq_len, embed_size]

        conv_results = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]  # Apply convs

        pooled_results = [torch.max(conv, dim=2)[0] for conv in conv_results]  # Max-pooling

        x = torch.cat(pooled_results, dim=1)  # Concatenate results of all kernel sizes

        x = self.dropout(x)

        x = self.fc(x)

        return x





# Load Emotion_final.csv dataset

emotion_data = pd.read_csv('/kaggle/input/emotions-in-text/Emotion_final.csv')



# Clean the text in the emotion data

def clean_text(text):

    text = str(text).lower()  # Convert to lowercase

    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters

    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces

    return text



emotion_data['Text'] = emotion_data['Text'].apply(clean_text)



# Load synthetic_data.csv dataset

synthetic_data = pd.read_csv('/kaggle/input/emotional-dataset-v2/synthetic_dataset.csv')



# Clean the text in the synthetic data

synthetic_data['Text'] = synthetic_data['Text'].apply(clean_text)



# Concatenate the original and synthetic datasets

combined_data = pd.concat([emotion_data, synthetic_data], ignore_index=True)



# Shuffle the combined dataset

combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)



# Encoding the labels

label_encoder = LabelEncoder()

combined_data['Emotion'] = label_encoder.fit_transform(combined_data['Emotion'])



# Tokenization: You could use pre-trained embeddings, or simply map words to indices. Here, we'll map to indices.



vocab = set(" ".join(combined_data['Text']).split())  # Unique words in the dataset

word2idx = {word: idx + 1 for idx, word in enumerate(vocab)}  # Map each word to an index

word2idx['<PAD>'] = 0  # Padding token



def text_to_tensor(text, word2idx, max_len=128):

    tokens = text.split()

    token_indices = [word2idx.get(token, 0) for token in tokens]  # Convert tokens to indices

    if len(token_indices) < max_len:

        token_indices.extend([0] * (max_len - len(token_indices)))  # Padding

    else:

        token_indices = token_indices[:max_len]  # Truncate if longer than max_len

    return torch.tensor(token_indices)



# Custom Dataset class for CNN

class TextDataset(Dataset):

    def __init__(self, texts, labels, word2idx, max_len=128):

        self.texts = texts.tolist()

        self.labels = labels.tolist()

        self.word2idx = word2idx

        self.max_len = max_len



    def __len__(self):

        return len(self.texts)



    def __getitem__(self, idx):

        text = self.texts[idx]

        label = int(self.labels[idx])  # Ensure label is converted to int

        text_tensor = text_to_tensor(text, self.word2idx, self.max_len)

        return {'text': text_tensor, 'label': torch.tensor(label, dtype=torch.long)}





# Hyperparameters

num_clients = 3

epochs = 100

learning_rate = 1e-3

#epsilon = 1.0

subset_size = 500

batch_size = 16

embed_size = 100  # Use a fixed embedding size

kernel_sizes = [3, 4, 5]  # Example kernel sizes for the CNN

num_filters = 128  # Number of filters for each kernel size



# Federated Learning - Split data for each client

clients_data = []

for _ in range(num_clients):

    client_data = combined_data.sample(frac=0.2, random_state=42).reset_index(drop=True).head(subset_size)

    clients_data.append(client_data)



# Train each client locally

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models = []

train_loss_history = [[] for _ in range(num_clients)]

train_acc_history = [[] for _ in range(num_clients)]



for i in range(num_clients):

    client_data = clients_data[i]

    train_texts, test_texts, train_labels, test_labels = train_test_split(client_data['Text'], client_data['Emotion'], test_size=0.2, random_state=42)

    assert len(train_texts) > 0 and len(test_texts) > 0, "Train-test split resulted in empty sets"



    train_dataset = TextDataset(train_texts, train_labels, word2idx)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



    model = CNN_Text(vocab_size=len(word2idx), embed_size=embed_size, num_classes=len(label_encoder.classes_), kernel_sizes=kernel_sizes, num_filters=num_filters).to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)



    for epoch in range(epochs):

        model.train()

        total_loss = 0

        correct = 0

        total = 0

        for batch in train_loader:

            texts = batch['text'].to(device)

            labels = batch['label'].to(device)



            optimizer.zero_grad()

            outputs = model(texts)

            loss = nn.CrossEntropyLoss()(outputs, labels)



            loss.backward()

            optimizer.step()



            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)

            total += labels.size(0)

            correct += (predicted == labels).sum().item()



        avg_loss = total_loss / len(train_loader)

        accuracy = correct / total

        train_loss_history[i].append(avg_loss)

        train_acc_history[i].append(accuracy)

        print(f"Client {i}, Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {accuracy}")



    models.append(model)



# Aggregation Step

# Combine the weights of the models for global model

global_state_dict = {}

for key in models[0].state_dict().keys():

    global_state_dict[key] = sum(model.state_dict()[key] for model in models) / num_clients



# Update the global model with the aggregated weights

global_model = CNN_Text(vocab_size=len(word2idx), embed_size=embed_size, num_classes=len(label_encoder.classes_), kernel_sizes=kernel_sizes, num_filters=num_filters).to(device)

global_model.load_state_dict(global_state_dict)





# Split the combined data into train and test datasets

train_texts, test_texts, train_labels, test_labels = train_test_split(combined_data['Text'], combined_data['Emotion'], test_size=0.2, random_state=42)



# Create a DataLoader for the test set

test_dataset = TextDataset(test_texts, test_labels, word2idx)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



# Evaluate the global CNN model on the test set (original test data)

global_model.eval()  # Set the model to evaluation mode



predictions = []

true_labels = []



# Evaluate on the test set

with torch.no_grad():

    for batch in test_loader:  # Use test_loader instead of original_test_loader

        texts = batch['text'].to(device)

        labels = batch['label'].to(device)

        outputs = global_model(texts)

        

        # Get predicted class labels

        _, predicted = torch.max(outputs, dim=1)

        

        predictions.extend(predicted.cpu().numpy())  # Convert to numpy array and add to predictions

        true_labels.extend(labels.cpu().numpy())     # Add true labels to the list



# Calculate the metrics

accuracy = accuracy_score(true_labels, predictions)

precision = precision_score(true_labels, predictions, average='weighted')

recall = recall_score(true_labels, predictions, average='weighted')

f1 = f1_score(true_labels, predictions, average='weighted')

cm = confusion_matrix(true_labels, predictions)



# Print the results

print(f"Forecast Test Accuracy: {accuracy:.4f}")

print(f"Forecast Test Precision: {precision:.4f}")

print(f"Forecast Test Recall: {recall:.4f}")

print(f"Forecast Test F1 Score: {f1:.4f}")

print("Forecast Confusion Matrix:")

print(cm)




In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Binarize the true labels for multi-class ROC curve
true_labels_bin = label_binarize(true_labels, classes=np.arange(len(label_encoder.classes_)))

# Evaluate on the test set and calculate probabilities (softmax)
model.eval()  # Set the model to evaluation mode

predictions_prob = []  # To store predicted probabilities

with torch.no_grad():
    for batch in test_loader:
        texts = batch['text'].to(device)
        labels = batch['label'].to(device)

        outputs = global_model(texts)

        # Apply softmax to get probabilities
        softmax_outputs = torch.nn.functional.softmax(outputs, dim=1)

        predictions_prob.extend(softmax_outputs.cpu().numpy())  # Store predicted probabilities

# Convert predictions_prob into a numpy array
predictions_prob = np.array(predictions_prob)

# Compute the ROC curve and AUC for each class
fpr, tpr, roc_auc = {}, {}, {}

for i in range(len(label_encoder.classes_)):
    fpr[i], tpr[i], _ = roc_curve(true_labels_bin[:, i], predictions_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve for each class
plt.figure(figsize=(10, 8))

for i in range(len(label_encoder.classes_)):
    plt.plot(fpr[i], tpr[i], label=f'Class {label_encoder.classes_[i]} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (chance level)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc='lower right')

plt.show()


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

# Load Emotion_final.csv dataset
emotion_data = pd.read_csv('/kaggle/input/emotions-in-text/Emotion_final.csv')

# Clean the text in the emotion data
def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    return text

emotion_data['Text'] = emotion_data['Text'].apply(clean_text)

# Load synthetic_data.csv dataset
synthetic_data = pd.read_csv('/kaggle/input/emotional-dataset-v2/synthetic_dataset.csv')

# Clean the text in the synthetic data
synthetic_data['Text'] = synthetic_data['Text'].apply(clean_text)

# Concatenate the original and synthetic datasets
combined_data = pd.concat([emotion_data, synthetic_data], ignore_index=True)

# Shuffle the combined dataset
combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Hyperparameters
num_clients = 3
epochs = 50
learning_rate = 1e-5
epsilon = 1.0
subset_size = 500
batch_size = 16

# Federated Learning - Split data for each client
clients_data = []
for _ in range(num_clients):
    client_data = combined_data.sample(frac=0.2, random_state=42).reset_index(drop=True).head(subset_size)
    clients_data.append(client_data)

# Encoding the labels
label_encoder = LabelEncoder()
for i in range(num_clients):
    clients_data[i]['Emotion'] = label_encoder.fit_transform(clients_data[i]['Emotion'])

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Custom Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts.tolist()  # Convert to list to ensure correct indexing
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(text, add_special_tokens=True, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Train each client locally
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = []
scaler = GradScaler()
train_loss_history = [[] for _ in range(num_clients)]
train_acc_history = [[] for _ in range(num_clients)]

for i in range(num_clients):
    client_data = clients_data[i]
    train_texts, test_texts, train_labels, test_labels = train_test_split(client_data['Text'], client_data['Emotion'], test_size=0.2, random_state=42)
    assert len(train_texts) > 0 and len(test_texts) > 0, "Train-test split resulted in empty sets"
    
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_)).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            with autocast():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

            _, predicted = torch.max(logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = total_loss / len(train_loader)
        accuracy = correct / total
        train_loss_history[i].append(avg_loss)
        train_acc_history[i].append(accuracy)
        print(f"Client {i}, Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {accuracy}")

    models.append(model)

# Aggregation Step
# Combine the weights of the models for global model
global_state_dict = {}
for key in models[0].state_dict().keys():
    global_state_dict[key] = sum(model.state_dict()[key] for model in models) / num_clients

# Update the global model with the aggregated weights
global_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_)).to(device)
global_model.load_state_dict(global_state_dict)

# Evaluate the global model on both original and synthetic test sets together
original_test_texts = emotion_data['Text']
original_test_labels = label_encoder.transform(emotion_data['Emotion'])
original_test_dataset = TextDataset(original_test_texts, original_test_labels, tokenizer)
original_test_loader = DataLoader(original_test_dataset, batch_size=batch_size)

synthetic_test_texts = synthetic_data['Text']
synthetic_test_labels = label_encoder.transform(synthetic_data['Emotion'])
synthetic_test_dataset = TextDataset(synthetic_test_texts, synthetic_test_labels, tokenizer)
synthetic_test_loader = DataLoader(synthetic_test_dataset, batch_size=batch_size)

# Function to calculate ROC AUC score
def calculate_roc_auc(model, data_loader, device, num_labels):
    model.eval()
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Convert logits to probabilities (softmax)
            probs = torch.nn.functional.softmax(logits, dim=1)

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

    all_labels = np.concatenate(all_labels, axis=0)
    all_probs = np.concatenate(all_probs, axis=0)

    # Binarize labels for multi-class ROC AUC
    all_labels_bin = label_binarize(all_labels, classes=np.arange(num_labels))
    
    # Compute ROC AUC score
    roc_auc = roc_auc_score(all_labels_bin, all_probs, average='macro', multi_class='ovr')
    return roc_auc

# Calculate ROC AUC for the global model on the combined test sets
roc_auc = calculate_roc_auc(global_model, synthetic_test_loader, device, len(label_encoder.classes_))
print(f"ROC AUC Score on the synthetic test set: {roc_auc:.4f}")


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import torch
import numpy as np

# Binarize the true labels for multi-class ROC curve
true_labels_bin = label_binarize(original_test_labels, classes=np.arange(len(label_encoder.classes_)))

# Evaluate on the test set and calculate probabilities (softmax)
global_model.eval()  # Set the model to evaluation mode

predictions_prob = []  # To store predicted probabilities

# Create DataLoader for original and synthetic test sets
original_test_dataset = TextDataset(original_test_texts, original_test_labels, tokenizer)
original_test_loader = DataLoader(original_test_dataset, batch_size=batch_size)

with torch.no_grad():
    for batch in original_test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Get model outputs (logits)
        outputs = global_model(input_ids, attention_mask=attention_mask)

        # Apply softmax to get probabilities
        softmax_outputs = torch.nn.functional.softmax(outputs.logits, dim=1)

        predictions_prob.extend(softmax_outputs.cpu().numpy())  # Store predicted probabilities

# Convert predictions_prob into a numpy array
predictions_prob = np.array(predictions_prob)

# Compute the ROC curve and AUC for each class
fpr, tpr, roc_auc = {}, {}, {}

for i in range(len(label_encoder.classes_)):
    fpr[i], tpr[i], _ = roc_curve(true_labels_bin[:, i], predictions_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve for each class
plt.figure(figsize=(10, 8))

for i in range(len(label_encoder.classes_)):
    plt.plot(fpr[i], tpr[i], label=f'Class {label_encoder.classes_[i]} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (chance level)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc='lower right')

plt.show()


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

# Load Emotion_final.csv dataset
emotion_data = pd.read_csv('/kaggle/input/emotions-in-text/Emotion_final.csv')

# Clean the text in the emotion data
def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    return text

emotion_data['Text'] = emotion_data['Text'].apply(clean_text)

# Load synthetic_data.csv dataset
synthetic_data = pd.read_csv('/kaggle/input/emotional-dataset-v2/synthetic_dataset.csv')
synthetic_data['Text'] = synthetic_data['Text'].apply(clean_text)

# Concatenate the original and synthetic datasets
combined_data = pd.concat([emotion_data, synthetic_data], ignore_index=True)
combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Hyperparameters
num_clients = 3  # Increase for more diverse federated learning
epochs = 100  # Keep, but implement early stopping based on validation loss
learning_rate = 5e-4  # Start with a lower learning rate, consider scheduling decay
subset_size = 1000  # Increase for more diverse data sampling
batch_size = 128  # Larger batch size for faster convergence
embedding_dim = 256  # Increase for capturing more complex patterns
hidden_dim = 256  # Increase for more model capacity


# Federated Learning - Split data for each client
clients_data = []
for _ in range(num_clients):
    client_data = combined_data.sample(frac=0.2, random_state=42).reset_index(drop=True).head(subset_size)
    clients_data.append(client_data)

# Encoding the labels
label_encoder = LabelEncoder()
for i in range(num_clients):
    clients_data[i]['Emotion'] = label_encoder.fit_transform(clients_data[i]['Emotion'])

# Tokenizer function for splitting words
def tokenize(text):
    return text.split()

# Building a vocabulary
all_words = set(word for text in combined_data['Text'] for word in tokenize(text))
word_to_index = {word: i+1 for i, word in enumerate(all_words)}  # Reserve index 0 for padding

# Custom Dataset class
# class TextDataset(Dataset):
#     def __init__(self, texts, labels, max_length=128):
#         self.texts = texts.tolist()
#         self.labels = labels.tolist()
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         label = int(self.labels[idx])
#         tokenized = [word_to_index.get(word, 0) for word in tokenize(text)]
#         padding = [0] * (self.max_length - len(tokenized))
#         input_ids = tokenized[:self.max_length] + padding if len(tokenized) < self.max_length else tokenized[:self.max_length]
#         return {
#             'input_ids': torch.tensor(input_ids, dtype=torch.long),
#             'labels': torch.tensor(label, dtype=torch.long)
#         }

class TextDataset(Dataset):
    def __init__(self, texts, labels, word_to_index, max_length=128):
        self.texts = texts.tolist()  # Ensure the texts are in list format
        self.labels = labels.tolist()  # Ensure labels are in list format
        self.word_to_index = word_to_index  # Ensure that word_to_index is passed
        self.max_length = max_length  # Ensure max_length is assigned properly

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = int(self.labels[idx])

        # Tokenize the text
        tokenized = [self.word_to_index.get(word, 0) for word in tokenize(text)]

        # Apply padding
        padding = [0] * (self.max_length - len(tokenized))  # Add padding
        input_ids = tokenized[:self.max_length] + padding if len(tokenized) < self.max_length else tokenized[:self.max_length]

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# BiGRU model definition
class BiGRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, max_length):
        super(BiGRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim, padding_idx=0)
        self.bigru = nn.GRU(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h = self.bigru(x)
        h = torch.cat((h[-2, :, :], h[-1, :, :]), dim=1)  # Concatenate final forward and backward hidden states
        return self.fc(h)

# Train each client locally
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = []
train_loss_history = [[] for _ in range(num_clients)]
train_acc_history = [[] for _ in range(num_clients)]

# for i in range(num_clients):
#     # Model and optimizer setup
#     model = BiGRUModel(len(word_to_index), embedding_dim, hidden_dim, len(label_encoder.classes_), max_length=128).to(device)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         correct = 0
#         total = 0
#         for batch in train_loader:
#             input_ids = batch['input_ids'].to(device)
#             labels = batch['labels'].to(device)

#             optimizer.zero_grad()
#             outputs = model(input_ids)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()

#             _, predicted = torch.max(outputs, dim=1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#         avg_loss = total_loss / len(train_loader)
#         accuracy = correct / total
#         train_loss_history[i].append(avg_loss)
#         train_acc_history[i].append(accuracy)
#         print(f"Client {i}, Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {accuracy:.4f}")

#     models.append(model)

# Create DataLoader for each client's data
for i in range(num_clients):
    # Create the dataset for the client
    train_dataset = TextDataset(clients_data[i]['Text'], clients_data[i]['Emotion'], word_to_index=word_to_index)
    
    # Create DataLoader for this client's dataset
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Model and optimizer setup
    model = BiGRUModel(len(word_to_index), embedding_dim, hidden_dim, len(label_encoder.classes_), max_length=128).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = total_loss / len(train_loader)
        accuracy = correct / total
        train_loss_history[i].append(avg_loss)
        train_acc_history[i].append(accuracy)
        print(f"Client {i}, Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {accuracy:.4f}")

    models.append(model)


# Aggregation Step: Averaging the weights of the models for a global model
global_state_dict = {}
for key in models[0].state_dict().keys():
    global_state_dict[key] = sum(model.state_dict()[key] for model in models) / num_clients

global_model = BiGRUModel(len(word_to_index), embedding_dim, hidden_dim, len(label_encoder.classes_), max_length=128).to(device)
global_model.load_state_dict(global_state_dict)

# Evaluate the global model
def evaluate_model(model, dataset):
    model.eval()
    data_loader = DataLoader(dataset, batch_size=batch_size)
    correct, total = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# original_test_dataset = TextDataset(emotion_data['Text'], label_encoder.transform(emotion_data['Emotion']))
# synthetic_test_dataset = TextDataset(synthetic_data['Text'], label_encoder.transform(synthetic_data['Emotion']))
# overall_accuracy = (evaluate_model(global_model, original_test_dataset) + evaluate_model(global_model, synthetic_test_dataset)) / 2
# print(f"Overall Test Accuracy: {overall_accuracy:.4f}")

# Create test datasets with the 'word_to_index' parameter
original_test_dataset = TextDataset(emotion_data['Text'], label_encoder.transform(emotion_data['Emotion']), word_to_index=word_to_index)
synthetic_test_dataset = TextDataset(synthetic_data['Text'], label_encoder.transform(synthetic_data['Emotion']), word_to_index=word_to_index)

# Evaluate the global model
overall_accuracy = (evaluate_model(global_model, original_test_dataset) + evaluate_model(global_model, synthetic_test_dataset)) / 2
print(f"Overall Test Accuracy: {overall_accuracy:.4f}")


# Plot the training loss and accuracy
plt.figure(figsize=(12, 6))
for i in range(num_clients):
    plt.subplot(2, num_clients, i+1)
    plt.plot(range(epochs), train_loss_history[i], label=f'Client {i} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Client {i} Training Loss')
    plt.legend()

    plt.subplot(2, num_clients, num_clients+i+1)
    plt.plot(range(epochs), train_acc_history[i], label=f'Client {i} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'Client {i} Training Accuracy')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import torch
import numpy as np

# Binarize the true labels for multi-class ROC curve
# Assuming 'original_test_labels' are the true labels of your test set
true_labels_bin = label_binarize(original_test_labels, classes=np.arange(len(label_encoder.classes_)))

# Evaluate on the test set and calculate probabilities (softmax)
global_model.eval()  # Set the model to evaluation mode

predictions_prob = []  # To store predicted probabilities

# Create DataLoader for original test set
original_test_dataset = TextDataset(original_test_texts, original_test_labels, word_to_index=word_to_index)
original_test_loader = DataLoader(original_test_dataset, batch_size=batch_size)

with torch.no_grad():
    for batch in original_test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # Get model outputs (logits)
        outputs = global_model(input_ids)

        # Apply softmax to get probabilities
        softmax_outputs = torch.nn.functional.softmax(outputs, dim=1)

        predictions_prob.extend(softmax_outputs.cpu().numpy())  # Store predicted probabilities

# Convert predictions_prob into a numpy array
predictions_prob = np.array(predictions_prob)

# Compute the ROC curve and AUC for each class
fpr, tpr, roc_auc = {}, {}, {}

for i in range(len(label_encoder.classes_)):
    fpr[i], tpr[i], _ = roc_curve(true_labels_bin[:, i], predictions_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve for each class
plt.figure(figsize=(10, 8))

for i in range(len(label_encoder.classes_)):
    plt.plot(fpr[i], tpr[i], color='tab:blue', lw=2, label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal line (random classifier)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Multi-Class')
plt.legend(loc='lower right')
plt.show()
